# Delta Lake - Consultas SQL no GovBR

Este notebook permite consultar os dados como um banco relacional usando Delta Lake.

**✅ CORRIGIDO:** Usa `configure_spark_with_delta_pip` e inclui pacotes S3A

In [ ]:
# Instalar dependências
!pip install -q delta-spark pyspark s3fs boto3

In [ ]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'

from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

# Configurações MinIO
MINIO_ENDPOINT = "ch8ai-minio.l6zv5a.easypanel.host"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "1q2w3e4r"
BUCKET_NAME = "govbr"

# Configurar Spark com Delta Lake e S3A
# IMPORTANTE: spark.jars.packages faz o Spark baixar automaticamente os JARs necessários
builder = SparkSession.builder \
    .appName("GovBR Delta Lake") \
    .config("spark.jars.packages", "io.delta:delta-spark_2.13:4.0.0,org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.endpoint", f"https://{MINIO_ENDPOINT}") \
    .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.master", "local[*]")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

print("✅ Spark Session criada com Delta Lake!")
print(f"✅ Versão Spark: {spark.version}")

## Registrar Tabelas Delta

In [ ]:
# Registrar todas as tabelas Delta
s3_base = f"s3a://{BUCKET_NAME}/delta"

tables = {
    "bronze_municipios": f"{s3_base}/bronze/bronze_municipios",
    "bronze_estados": f"{s3_base}/bronze/bronze_estados",
    "bronze_bpc": f"{s3_base}/bronze/bronze_bpc",
    "prata_dim_municipios": f"{s3_base}/prata/prata_dim_municipios",
    "prata_dim_estados": f"{s3_base}/prata/prata_dim_estados",
    "prata_fato_bpc": f"{s3_base}/prata/prata_fato_bpc",
    "ouro_dim_municipios": f"{s3_base}/ouro/ouro_dim_municipios",
    "ouro_dim_estados": f"{s3_base}/ouro/ouro_dim_estados",
    "ouro_fato_bpc": f"{s3_base}/ouro/ouro_fato_bpc"
}

for table_name, delta_path in tables.items():
    try:
        spark.sql(f"""
            CREATE TABLE IF NOT EXISTS {table_name}
            USING DELTA
            LOCATION '{delta_path}'
        """)
        print(f"✅ {table_name} registrada")
    except Exception as e:
        print(f"⚠️  {table_name}: {e}")

## Consultas SQL

In [ ]:
# Listar todas as tabelas
spark.sql("SHOW TABLES").show()

In [ ]:
# Consulta 1: Estados com população
spark.sql("""
    SELECT uf_sigla, uf_nome, populacao 
    FROM ouro_dim_estados 
    WHERE populacao IS NOT NULL
    ORDER BY populacao DESC 
    LIMIT 10
""").show(truncate=False)